# **Predict Cancer Mortality Rates in US Counties**

The provided dataset comprises data collected from multiple counties in the US. The regression task for this assessment is to predict cancer mortality rates in "unseen" US counties, given some training data. The training data ('Training_data.csv') comprises various features/predictors related to socio-economic characteristics, amongst other types of information for specific counties in the country. The corresponding target variables for the training set are provided in a separate CSV file ('Training_data_targets.csv'). Use the notebooks provided for lab sessions throughout this module to provide solutions to the exercises listed below. Throughout all exercises, text describing your code and answering any questions included in the exercise descriptions should be provided as part of your submitted solution. (Total Marks for this Assessment is 40)

Note - We also provide an example test data set ('Test_data_example.csv' and 'Test_data_example_targets.csv'). This is just an example of the final test set (which will not be provided to you) that will be used to evaluate your solutions when your submitted solutions are being marked. Part of this assessment requires you to write an inference script that evaluates the regression models you have trained on the final test data set such that we are able to run the inference script ourselves on the test data (you can use the example test data to verify that it works prior to submission).

The list of predictors/features available in this data set are described below:

**Data Dictionary**

avgAnnCount: Mean number of reported cases of cancer diagnosed annually

avgDeathsPerYear: Mean number of reported mortalities due to cancer

incidenceRate: Mean per capita (100,000) cancer diagoses

medianIncome: Median income per county 

popEst2015: Population of county 

povertyPercent: Percent of populace in poverty 

MedianAge: Median age of county residents 

MedianAgeMale: Median age of male county residents 

MedianAgeFemale: Median age of female county residents 

AvgHouseholdSize: Mean household size of county 

PercentMarried: Percent of county residents who are married 

PctNoHS18_24: Percent of county residents ages 18-24 highest education attained: less than high school 

PctHS18_24: Percent of county residents ages 18-24 highest education attained: high school diploma 

PctSomeCol18_24: Percent of county residents ages 18-24 highest education attained: some college 

PctBachDeg18_24: Percent of county residents ages 18-24 highest education attained: bachelor's degree 

PctHS25_Over: Percent of county residents ages 25 and over highest education attained: high school diploma 

PctBachDeg25_Over: Percent of county residents ages 25 and over highest education attained: bachelor's degree 

PctEmployed16_Over: Percent of county residents ages 16 and over employed 

PctUnemployed16_Over: Percent of county residents ages 16 and over unemployed 

PctPrivateCoverage: Percent of county residents with private health coverage 

PctPrivateCoverageAlone: Percent of county residents with private health coverage alone (no public assistance) 

PctEmpPrivCoverage: Percent of county residents with employee-provided private health coverage 

PctPublicCoverage: Percent of county residents with government-provided health coverage 

PctPubliceCoverageAlone: Percent of county residents with government-provided health coverage alone 

PctWhite: Percent of county residents who identify as White 

PctBlack: Percent of county residents who identify as Black 

PctAsian: Percent of county residents who identify as Asian 

PctOtherRace: Percent of county residents who identify in a category which is not White, Black, or Asian 

PctMarriedHouseholds: Percent of married households 

BirthRate: Number of live births relative to number of women in county 

In [ ]:
import os
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

## Define paths to the training data and targets files
training_data_path = 'Training_data.csv'
training_targets_path = 'Training_data_targets.csv'

pd.set_option('display.max_rows', None)

# **Exercise 1**

Read in the training data and targets files. The training data comprises features/predictors while the targets file comprises the targets (i.e. cancer mortality rates in US counties) you need to train models to predict. Plot histograms of all features to visualise their distributions and identify outliers. Do you notice any unusual values for any of the features? If so comment on these in the text accompanying your code. Compute correlations of all features with the target variable (across the data set) and sort them according the strength of correlations. Which are the top five features with strongest correlations to the targets? Plot these correlations using the scatter matrix plotting function available in pandas and comment on at least two sets of features that show visible correlations to each other. (5 marks)

In [ ]:
# read in training and target files as csv
training = pd.read_csv(training_data_path)
targets = pd.read_csv(training_targets_path)

# histograms for all features 
training.hist(bins=30, figsize=(20,15))

# identify outliers
# avgHouseholdSize has a small value which outlies from the rest of the data
# by examining the training dataset directly, I also noticed some very 
# implausible values for the median age variable. There were 26 records where 
# the median age was more than 300 - this obviously could not be true
# small bars, huge whitespace (range extends far), large scale for x axis - outliers exist inthe distribution, but not necessarily implausible e.g. predominantly black areas

combined = pd.merge(training, targets, left_index=True, right_index=True)
corr_matrix=combined.corr()
sortedCorrs = corr_matrix.reindex(corr_matrix.TARGET_deathRate.abs().sort_values(ascending=False).index)["TARGET_deathRate"]
print(sortedCorrs)

attributes=["TARGET_deathRate","PctBachDeg25_Over","incidenceRate","PctPublicCoverageAlone","medIncome","povertyPercent"]

scatter_matrix(combined[attributes],figsize=(12,8))

plt.savefig('scatter_matrix.png')

# PctPublicCoverageAlone - PovertyPercent
# When plotting PovertyPercent against PctPublicCoverageAlone, there is a clear and strong positive correlation between the features.
# As the value for PovertyPercent increases i.e., the percentage of the populace in poverty increase, the value for PctPublicCoverageAlone also increases i.e., the percentage of the 
# population with government funded healthcare also increases. This makes sense intuitively, as you'd expect there to be more
# people who require government funding for healthcare in areas of higher poverty.

# PctBachDeg25_Over - medIncome
# When plotting PctBachDeg25_Over (the percentage of the population in a county aged 25< with a degree) against the medianIncome 
# in the respective county, there is a clear positive correlation between the two features. This also makes sense intuitively, as you'd 
# expect the average salary to increase given that someone has a degree and hence, where there is a higher percentage of people
# with degrees, you find a higher median income value for that county. 


In [ ]:
# for my own interest, to see how other features correlate with each other
attributes=["TARGET_deathRate","incidenceRate","PctPublicCoverageAlone","medIncome","povertyPercent", "avgAnnCount", "avgDeathsPerYear", "popEst2015"]

scatter_matrix(combined[attributes],figsize=(12,8), alpha=0.3)
plt.savefig('scatter_matrix_full.png')

In [ ]:
# calc ranges
columns = combined.columns.values.tolist()

for column in columns:
  range = combined[column].max() - combined[column].min()
  print(f"column: {column}, range: {range}")

# median age range shows there are outliers in median age

In [ ]:
# show maximums
columns = combined.columns.values.tolist()

for column in columns:
  max = combined[column].max()
  print(f"column: {column}, max: {max}")

# median age definitely has outliers

# **Exercise 2**

Create an ML pipeline using scikit-learn (as demonstrated in the lab notebooks) to pre-process the training data. (3 marks)

In [ ]:
# remove the rows with unrealistic median age values and avgHouseholdSize values
from sklearn.base import BaseEstimator

missing_columns = ['PctSomeCol18_24', 'PctEmployed16_Over', 'PctPrivateCoverageAlone']

class RemoveImplausibleValues(BaseEstimator):
    
    def fit(self,X, y = None):
        return self
    
    def transform(self, X):
        x = X.copy(deep=True)
        # x = x[~((x['MedianAge'] > 300) | (x['AvgHouseholdSize'] <1))]
        # for column in x.columns:
        #     if x[column].isnull().sum() > 0:
        #         print(column)
        #         missing_columns.append(column)
        #         x.drop(column, axis=1, inplace=True)
        x.drop(missing_columns, axis=1, inplace=True)


        return x


# print(len(combined))
# combined = combined[~((combined['MedianAge'] > 300) | (combined['AvgHouseholdSize'] <1))]
# print(len(combined))


In [ ]:
# before splitting the data, remove records with implausible values
# create scikit learn pipeline
from sklearn.pipeline import Pipeline

initial_pipeline = Pipeline([
    ('RemoveImplausibleValues',RemoveImplausibleValues())
])

print(len(combined))
combined_ppln = initial_pipeline.fit_transform(combined).reset_index(drop=True)
# combined_ppln["TARGET_deathRate"].head()
print(len(combined_ppln))

In [ ]:
print(combined_ppln.info())

In [ ]:
# split data to train and test set
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

def stratified_split(df, feature):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=42)
    
    # returns 2 sets of indexes for test and train
    # hence, .loc is used on the dataset df to retrieve the corresponding records
    for train_index, test_index in split.split(df,df[feature]):
        strat_train_set = df.loc[train_index]
        strat_test_set = df.loc[test_index]
    
    for set_ in (strat_train_set, strat_test_set):
        set_.drop((feature),axis=1,inplace=True)
        
   
    return strat_train_set, strat_test_set


def shuffle_split(df, target):
    split = ShuffleSplit(n_splits=1, test_size=0.2,random_state=42)
    
    # returns 2 sets of indexes for test and train
    # hence, .loc is used on the dataset df to retrieve the corresponding records
    for train_index, test_index in split.split(df,df[target]):
        train_set = df.loc[train_index]
        test_set = df.loc[test_index]
    
#     for set_ in (strat_train_set, strat_test_set):
#         set_.drop((target),axis=1,inplace=True)
        
   
    return train_set, test_set


train_set, test_set = shuffle_split(combined_ppln, 'TARGET_deathRate')

training=train_set.drop("TARGET_deathRate",axis=1)
training_labels=train_set["TARGET_deathRate"].copy()

testing = test_set.drop("TARGET_deathRate",axis=1)
testing_labels= test_set["TARGET_deathRate"].copy()



In [ ]:
len(training), len(testing)

In [ ]:
combined.info()
# NOTE: PctSomeCol18_24, PctEmployed16_Over,PctPrivateCoverageAlone all contain 
# less than 2438 non-null values i.e. have null values which require imputing

In [ ]:
# it seems some columns (PctSomeCol18_24, PctEmployed16_Over, PctPrivateCoverageAlone) have null values - 
# let's validate this
# print(combined['PctSomeCol18_24'].isnull().any())
# print(combined['PctEmployed16_Over'].isnull().any())
# print(combined['PctPrivateCoverageAlone'].isnull().any())


In [ ]:
# impute - currently using median!
# from sklearn.impute import SimpleImputer

# imputer=SimpleImputer(strategy="median")
# # fit
# imputer.fit(combined)
# # array of size 32, with median value of each column
# imputer.statistics_ 
# # transform
# x = imputer.transform(combined)
# x.shape
# # turn np.array back into dataframe
# combined = pd.DataFrame(x,columns=combined.columns)
# combined.info()


***Q to self: IS MEDIAN THE BEST CHOICE TO IMPUTE WITH? TRY WITH MEAN AND SEE WHICH YIELDS BETTER RESULTS!!***

In [ ]:
# aggregate and add features


# normalise - make values 0 to 1 - used for neural networks


# Standardise - minus mean, divide by variance


# no encoding needed e.g. one hot encoding, since all variables are numerical


In [ ]:
# create scikit learn pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np

# could try iterative imputer
pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    ('std_scaler',StandardScaler())
])

training_ppln = pipeline.fit_transform(training)
print(len(training_ppln))
# type(combined_ppln), combined.columns
# turn it back into a df
# training_ppln = pd.DataFrame(data=training_ppln, columns=training.columns)
# print(training_ppln.info())

In [ ]:
training_ppln_df = pd.DataFrame(data=training_ppln, columns=training.columns)
training_ppln_df.head()


# **Exercise 3**

Fit linear regression models to the pre-processed data using: Ordinary least squares (OLS), Lasso and Ridge models. Choose suitable regularisation weights for Lasso and Ridge regression and include a description in text of how they were chosen. In your submitted solution make sure you set the values for the regularisation weights equal to those you identify from your experiment(s). Quantitatively compare your results from all three models and report the best performing one. Include code for all steps above. (10 marks)


In [ ]:
# use combined df
# split train test sets 80:20 - use stratified sampling based on a feature/variable?
# do linear regression on training set with cross validation 
# get metrics from cross validation and aggregate results (E.g. avg)
# to measure how effective selected hyperparameters are
# use gridsearchCV to automatically find the best hyper parameters for model


In [ ]:
some_data=testing
some_labels=testing_labels
some_data_prepared=pipeline.transform(some_data)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#OLS - Ordinary Least Squares

lin_reg=LinearRegression()
lin_reg.fit(training_ppln, training_labels)


In [ ]:
predicted = lin_reg.predict(some_data_prepared)
# print("Predictions:", predicted)
# print("Labels:", list(some_labels))
mse = mean_squared_error(list(some_labels), predicted)
rmse = np.sqrt(mse)
mae = mean_absolute_error(list(some_labels), predicted)
r2 = r2_score(list(some_labels), predicted)
print(f"mse: {mse}")
print(f"rmse: {rmse}")
print(f"mae: {mae}")
print(f"r2: {r2}")


In [ ]:
from sklearn.model_selection import GridSearchCV
lasso = Lasso()

#regularisation strength - alpha
param_grid = {'alpha': np.arange(1,100,1)}


# Create a GridSearchCV object - using 5 folds, could try 10 folds too or more
grid_search = GridSearchCV(lasso, param_grid, cv=10, scoring='neg_mean_squared_error')

# Fit the model to the data using the grid search object
grid_search.fit(training_ppln, training_labels)

# Get the best regularization weight
best_alpha = grid_search.best_params_['alpha']
best_alpha

***BEST LASSO REGULARISATION WEIGHT:***
The best value of alpha found for Lasso was 0.0415. I found this value for alpha by starting with a relatively large range of values for alpha (0.001 to 10) and iteratively narrowing down the range to  converge on a more optimal value for alpha.
The best values found for alpha after each iteration were \[0.01, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04,0.042, 0.041, 0.0415\]

So, the value I decided to use was 0.0415 for alpha.

In [ ]:
lasso = Lasso(alpha=0.0415).fit(training_ppln, training_labels)

In [ ]:
predicted = lasso.predict(some_data_prepared)
# print("Predictions:", predicted)
# print("Labels:", list(some_labels))
mse = mean_squared_error(list(some_labels), predicted)
rmse = np.sqrt(mse)
mae = mean_absolute_error(list(some_labels), predicted)
r2 = r2_score(list(some_labels), predicted)

print(f"mse: {mse}")
print(f"rmse: {rmse}")
print(f"mae: {mae}")
print(f"r2: {r2}")


In [ ]:
ridge = Ridge()

#regularisation strength - alpha
param_grid = {'alpha': [13.34, 13.35, 13.6]}


# Create a GridSearchCV object - using 5 folds, could try 10 folds too or more
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model to the data using the grid search object
grid_search.fit(training_ppln, training_labels)

# Get the best regularization weight
best_alpha = grid_search.best_params_['alpha']
best_alpha

***BEST RIDGE REGULARISATION WEIGHT:***
The best value of alpha found for Lasso was 13.35. I found this value for alpha by starting with a relatively large range of values for alpha (0.001 to 10) and iteratively narrowing down the range to  converge on a more optimal value for alpha.
The best values found for alpha after each iteration were \[10, 10, 15, 12.5, 13.75, 13, 13.5, 13.25, 13.4, 13.35, 13.35, 13.35\]

In [ ]:
ridge = Ridge(alpha=13.35).fit(training_ppln, training_labels)

In [ ]:
predicted = ridge.predict(some_data_prepared)
# print("Predictions:", predicted)
# print("Labels:", list(some_labels))
mse = mean_squared_error(list(some_labels), predicted)
rmse = np.sqrt(mse)
mae = mean_absolute_error(list(some_labels), predicted)
r2 = r2_score(list(some_labels), predicted)

print(f"mse: {mse}")
print(f"rmse: {rmse}")
print(f"mae: {mae}")
print(f"r2: {r2}")


# **Exercise 4**

Use Lasso regression and the best regularisation weight identified from Exercise 3 to identify the five most important/relevant features for the provided data set and regression task. Report what these are desceding order of their importance. (5 marks)

In [ ]:
weights = np.abs(lasso.coef_)

# sortedCorrs = corr_matrix.reindex(corr_matrix.TARGET_deathRate.abs().sort_values(ascending=False).index)["TARGET_deathRate"]

features_weights = sorted(dict(zip(training_ppln_df.columns, weights)).items(), key=lambda x: x[1], reverse=True)

feature_weights = pd.DataFrame(features_weights[:5], columns=['Feature', 'Weight'])

display(feature_weights)

# **Exercise 5**

Fit a Random Forest regression model to the training data and quantitatively evaluate and compare the Random Forest regression model with the best linear regression model identified from Exercise 3. Report which model provides the best results. Next, report the top five most important/relevant features for the provided data set and regression task identified using the Random Forest model. Comment on how these compare with the features identified from Lasso regression? (14 marks)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=42)
parameters = {
  'n_estimators': [100],
  'max_depth': [10],
  "random_state": [21],
  "max_features" : [30]
}

forest = GridSearchCV(forest, parameters, scoring='neg_mean_squared_error', cv=10)
forest.fit(training, np.ravel(training_labels))
forest_predictions = forest.predict(testing)
forest = forest.best_estimator_
print(f"forest: {forest}")


In [ ]:
# metrics RMSE, MSE, MAE, R2
# predicted = forest.predict(some_data_prepared)
# print("Predictions:", predicted)
# print("Labels:", list(some_labels))
mse = mean_squared_error(list(some_labels), forest_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(list(some_labels), forest_predictions)
r2 = r2_score(list(some_labels), predicted)

print(f"mse: {mse}")
print(f"rmse: {rmse}")
print(f"mae: {mae}")
print(f"r2: {r2}")


In [ ]:
# compare with best linear one

In [ ]:
# show top 5 features from RF
relevance = forest.feature_importances_
features_weights = sorted(dict(zip(training_ppln_df.columns, relevance)).items(), key=lambda x: x[1], reverse=True)

feature_weights = pd.DataFrame(features_weights[:5], columns=['Feature', 'Weight'])

display(feature_weights)

# **Exercise 6**

Use the provided test example data ('Test_data_example.csv' and 'Test_data_example_targets.csv') to write an inference script to evaluate the best regression model identified from preceding exercises. First re-train the chosen regression model using all of the provided training data and test your predictions on the provided example test data. Note - the final evaluation of your submission will be done by replacing this example test data with held out (unseen) test data that is not provided to you. But the format of this "unseen" test data will be identical to the example test data provided to you. Use the code snippet provided below to prepare your inference script to predict targets for the unseen test data. (3 marks)

In [ ]:
## Read in the provided example test data
test_data_path = 'Test_data_example.csv'
test_targets_path ='Test_data_example_targets.csv'

test_data = pd.read_csv(test_data_path)
test_targets = pd.read_csv(test_targets_path)
## Retrain your chosen regression model here 
# For example: lin_reg = LinearRegression()
# lin_reg.fit(X_train,y_train) where X_train and y_train is provided training data
# Next write the lines of code required to predict on unseen test data and evaluate your predictions

In [ ]:
training_new = pd.read_csv(training_data_path)

# training_new_labels=combined_ppln["TARGET_deathRate"].copy()
# training_new=combined_ppln.drop("TARGET_deathRate",axis=1, inplace=True)

train_data_ppln = pipeline.fit_transform(combined_ppln)

test_data_ppln = initial_pipeline.fit_transform(test_data)
test_data_ppln = pipeline.fit_transform(test_data_ppln)

lasso_final = Lasso(alpha=0.0415)
lasso_final.fit(train_data_ppln, training_new_labels)
print(len(train_data_ppln[0]), len(training_new_labels))

print(len(test_data_ppln[0]))
final_predictions = lasso_final.predict(test_data_ppln)

print(len(test_targets), len(final_predictions))

mse = mean_squared_error((test_targets), final_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error((test_targets), final_predictions)
r2 = r2_score((test_targets), final_predictions)

print(f"mse: {mse}")
print(f"rmse: {rmse}")
print(f"mae: {mae}")
print(f"r2: {r2}")